# Inges notebook
This notebook takes normalized TSV files and reads them into a breach specific database. Before you run this make sure you update the constants below.

# Set constants

In [ ]:
# change these
CASE="DIVD-2021-00012"
SUB="set1"
CONTAINS_PASSWORDS=True
# Keep the same
IN_DIR="../NORMALIZED"
OUT_DIR="../LEAK_DB/"
OUT_DB=f"{OUT_DIR}/{CASE}-{SUB}.sqlite3"

# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

# Import data

In [ ]:
!ls $OUT_DIR


In [ ]:
#!rm $OUT_DIR/*

# Open DB (or create it)

In [ ]:
if not os.path.exists(OUT_DB) :
    create_leak_db(OUT_DB)
# Open database
conn = sqlite3.connect(OUT_DB)
cur = conn.cursor()

# Import records

In [ ]:
files=sorted(glob(f"{IN_DIR}/*.tsv"))

In [ ]:
files

In [ ]:
cur.execute("DELETE FROM 'entity'")
count=0
for file in files :
    df = pd.read_csv(file, sep='\t', index_col=False, 
        dtype={
            "username": str, "passwd" : str, "url": str, "extra_data": str }
    )
    for index, row in df.iterrows():
        if type(row["username"]) is str :
            user_ext = tldextract.extract(row["username"] )
            user_apex = f"{user_ext.domain}.{user_ext.suffix}"
        else:
            user_ext = None
            user_apex = None
        if type(row["url"]) is str:
            url = row["url"]
            url_ext = tldextract.extract(url)
            url_apex = f"{url_ext.domain}.{url_ext.suffix}"
        else:
            url = None
            url_ext = None
            url_apex = None
        if type(row["username"]) is str:
            username = row["username"]
        else:
            username = None
        if type(row["passwd"]) is str:
            passwd = row["passwd"]
            masked_passwd = mask_password(passwd)
        else:
            passwd = None
            masked_passwd = None
        if username and passwd and is_email(username):
            nml_hash_str = nml_hash(username, passwd)
        else:
            nml_hash_str = None
        cur.execute("""
            INSERT into 'entity' ( 
                username, masked_passwd, nml_hash, url, 
                email_apex, url_apex, ts_found, ts_leaked, 
                has_name, has_dob, has_addr, has_phone, has_cc, has_bankacc,
                has_ssn, has_ip, extra_data
            ) values (
                ?, ?, ?, ?, 
                ?, ?, ?, ?, 
                ?, ?, ?, ?, ?, ?, 
                ?, ?, ?
            )
        """ , (
            username, masked_passwd, nml_hash_str, url,
            user_apex, url_apex, row["ts_found"], row["ts_leaked"],
            row["has_name"], row["has_dob"], row["has_addr"], row["has_phone"], row["has_cc"], row["has_bankacc"],
            row["has_ssn"], row["has_ip"], row["extra_data"]
        ) )
        count=count+1
        if count % 1_000 == 0 :
            print(f"{count:,}", end="\r")
        if count % 10_000 == 0 :
            conn.commit()
conn.commit()

In [ ]:
conn.commit()